# Multihead Attention; wavelets


https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb

Lets test multiple attention heads, one for each input, with univariate and mutivariate data

Lets test wavelet threshold denoising

Due to a slip in the matrix profile function, the matrix profile was computed in reverse, this is adressed and computed normally in the 'Multihead Att_fwd; wavelets'

## Load data, functions

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import mean
from numpy import std
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import MultiHeadAttention, BatchNormalization, LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pywt
import matrixprofile as mp
from keras import Model
from keras.layers import dot
import datetime 
from datetime import timedelta
import math

In [2]:
def split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead):
    X, y = list(), list()
    example_count = int((len(sequence)/step_interval))
    for i in range(example_count):
        # find the end of this pattern
        end_ix = (i*step_interval) + n_steps_in
        out_start_ix = end_ix + n_step_lookahead -1
        out_end_ix = end_ix + n_steps_out + n_step_lookahead -1
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[(i*step_interval):end_ix], sequence[out_start_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#To demonstrate above function
sequence = range(0,13)
n_steps_in = 1
n_steps_in = 5
n_steps_out =1
step_interval =1
n_step_lookahead=5
split_sequence(sequence, n_steps_in, n_steps_out, step_interval, n_step_lookahead)

(array([[0, 1, 2, 3, 4],
        [1, 2, 3, 4, 5],
        [2, 3, 4, 5, 6],
        [3, 4, 5, 6, 7]]),
 array([[ 9],
        [10],
        [11],
        [12]]))

In [4]:
percentile_data = pd.read_csv (r'C:\Users/conal/Desktop/MCM/Practicum - Copy/data/block gas price percentile data.csv', header=0)
percentile_data['datetime'] = pd.to_datetime(percentile_data['block_timestamp'], format = '%Y-%m-%d %H:%M:%S UTC')

percentile_data = percentile_data.sort_values(by='datetime',ascending=False)
percentile_data = percentile_data.set_index('datetime')
percentile_data = percentile_data.resample('5T').mean()
percentile_data = percentile_data/1000000000

In [5]:
usage_data = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\ETH,gas,usage merged 11-26 to 05-26.csv', header=0)
usage_data['datetime'] = pd.to_datetime(usage_data['datetime'], format = '%Y-%m-%d %H:%M:%S')
usage_data = usage_data.set_index('datetime')

usage_data = usage_data.squeeze()
usage_data = usage_data.astype('float')
usage_data = usage_data.resample('5T').mean()

In [6]:
usage_data2 = pd.read_csv (r'C:\Users\conal\Desktop\MCM\Practicum - Copy\data\Contract counts 2021-11-26 to 2022-05-26.csv', header=0, index_col=0)
usage_data2['datetime'] = pd.to_datetime(usage_data2['block_timestamp'], format = '%Y-%m-%d %H:%M:%S') 
usage_data2 = usage_data2.set_index('datetime')
usage_data2 = usage_data2.drop(['block_timestamp'], axis=1)
usage_data2 = usage_data2.squeeze()
usage_data2 = usage_data2.astype('float')
usage_data2 = usage_data2.resample('5T').sum()

In [7]:
data = usage_data.merge(percentile_data, left_index=True, right_index=True)
data = data.merge(usage_data2, left_index=True, right_index=True)

Load data, datetime to index, downsample with left edge label, convert wei to gwei

In [1]:
def generate_training_val_examples(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    #we are only lookign to forecast the min gas price
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))

    
    return X_train, y_train, X_val, y_val, scaler



In [2]:
def LSTM_model():
    model = Sequential()
    model.add(LSTM(100, activation='tanh', input_shape=(n_steps_in, len(inputs))))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(100, activation='tanh', return_sequences=True))
    model.add(TimeDistributed(Dense(len(inputs))))
    model.compile(optimizer='adam', loss='mse')
    return model

In [3]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error


In [4]:
def descale_y_retrun_metrics(yhat, y_val2):
    #We will use validation data that has not had outleirs limited, will be a different min/max scaler as such
    dict_indexes=[]
    dict_dfs=[]
    for j in range(0, n_steps_out):
        RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
        for i in range(0, len(inputs)):  
            pred_descaled= (scaler.inverse_transform(yhat[:,j:j+1,:].reshape(yhat.shape[0], yhat.shape[2])))[:, i:i+1]
            groud_truth_descaled= ((scaler2.inverse_transform(y_val2[:,j:j+1,:].reshape(y_val2.shape[0], y_val2.shape[2]))))[:, i:i+1]
            RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
            MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
            MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
            MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
            R2 = r2_score(groud_truth_descaled, pred_descaled)
            RMSE_list.append(RMSE)
            MAE_list.append(MAE)
            MAPE_list.append(MAPE)
            R2_list.append(R2)
            MSE_list.append(MSE)
        metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=inputs)
        dict_dfs.append(metrics_df)
        dict_indexes.append('Lookahead' +str(j))
    metrics_dict = dict(zip(dict_indexes, dict_dfs))
    return metrics_dict

In [5]:
def generate_training_val_examples_univariate_output(data):
 
    #Filter inputs, standardize
    data =data[inputs]
    scaler = StandardScaler()
    data[inputs] = scaler.fit_transform(data[inputs])
    

    #Creat input:output examples
    data = data[start_date:end_date].to_numpy()
    X, y = split_sequence(data, n_steps_in, n_steps_out, step_interval, n_step_lookahead)
    
    X_train, X_val = np.split(X, [int(0.7 * len(X))])
    
    y_train, y_val = np.split(y, [int(0.7 * len(X))])

    
    #Reshape to 3D for LSTM, filter output to only the first input
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], len(inputs)))
    y_train =y_train.reshape((y_train.shape[0], y_train.shape[1], len(inputs)))[:,:,:1]
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], len(inputs)))
    y_val = y_val.reshape((y_val.shape[0], y_val.shape[1], len(inputs)))[:,:,:1]

    
    return X_train, y_train, X_val, y_val, scaler

In [6]:
def descale_y_retrun_metrics_univariate_y(yhat, y_val2):
    #reverts standard scaling, returns dictionary of metrics for single output, for all lookaheads
    dict_indexes=[]
    dict_dfs=[]
    RMSE_list, MAE_list, MAPE_list, R2_list, MSE_list = [],[],[],[],[]
    yhat_stretched= np.repeat(yhat[:,:,0],len(inputs)).reshape(yhat.shape[0], yhat.shape[1], len(inputs))
    y_val2_stretched= np.repeat(y_val2[:,:,0],len(inputs)).reshape(y_val2.shape[0], y_val2.shape[1], len(inputs))
    
    for j in range(0, n_steps_out):
        
        
        pred_descaled= (scaler.inverse_transform(yhat_stretched[:, j:j+1, :].reshape(yhat_stretched.shape[0], yhat_stretched.shape[2])))[:,:1]
        groud_truth_descaled= (scaler.inverse_transform(array([y_val2[ :, j:j+1,0].reshape(y_val2.shape[0])]*len(inputs)).transpose()))[:,:1]
        RMSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=False)
        MSE = mean_squared_error(groud_truth_descaled, pred_descaled, squared=True)
        MAE = mean_absolute_error(groud_truth_descaled, pred_descaled)
        MAPE = mean_absolute_percentage_error(groud_truth_descaled, pred_descaled)
        R2 = r2_score(groud_truth_descaled, pred_descaled)
        RMSE_list.append(RMSE)
        MAE_list.append(MAE)
        MAPE_list.append(MAPE)
        R2_list.append(R2)
        MSE_list.append(MSE)
    metrics_df = pd.DataFrame({'RMSE':RMSE_list, 'MSE':MSE_list, 'MAE':MAE_list, 'MAPE':MAPE_list, 'R2':R2_list}, index=range(1, (n_steps_out+1)))

 
    return metrics_df

In [7]:
def add_mp_reversed(data, window):
    #Given 3d array, add matrix profile of (x,y,0) as new dimension; new array has dimensiosn (x,y,z+1) 
    mp_list=[]
    for i in data[:,:,0]:
        profile = mp.compute(np.flip(i, axis=0), window, n_jobs=4)['mp']
        #we are padding the end of the sequence with the mean
        #matrix profile is always 1 full window size smalelr than input data
        mp_list.append(np.append(profile,([mean(profile)]*(data.shape[1]-len(profile)))))
        
    #concatenate matrix profile data with original    
    mp_array = np.array(mp_list).reshape(data.shape[0], data.shape[1])
    std_array = ((mp_array-mean(mp_array))/np.std(mp_array)).reshape(data.shape[0], data.shape[1],1)
    data = np.concatenate((data, std_array), axis=2)[:, window:, :]
    
    return data

In [8]:
def madev(d, axis=None):
    """ Mean absolute deviation of a signal """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [9]:
def wavelet_denoising(x, wavelet='db4', level=2, tmod=1):
    
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1/tmod) * madev(coeff[-level])
    uthresh = sigma * np.sqrt(3 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

In [10]:
def denoise_array(data, variable, wavelet, level, tmod):
    denoised_examples=[]
    for i in (data[:,:,variable]):
        
        denoised_examples.append(wavelet_denoising(i, wavelet=wavelet, level=level, tmod=tmod))
        denoised_array= np.array(denoised_examples)
        denoised_array= denoised_array.reshape(denoised_array.shape[0], denoised_array.shape[1], 1)
    return np.concatenate((denoised_array, data[:,:,1:]),axis=2)

## 2 layer, all variables, MP

In [21]:
def LSTM_model(): 
    
    #set up callback for best val loss model
    checkpoint_filepath='./cnn/checkpoint'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    
    n_hidden = 30
    input_train = Input(shape=(n_steps_in, X_train.shape[2]),name='input')
    output_train = Input(shape=( y_train.shape[1], y_train.shape[2]))




    enc_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('encoder' +str(i)))(input_train)

        decoder_input = RepeatVector(y_train.shape[1], name='repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('attention_dot'+str(i)))
        attention = Activation('softmax', name='attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='Context'+str(i))
        enc_head_list.append(context)
    enc_concat_attention = Concatenate(axis=2)(enc_head_list)
    
    
    
    dec_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('decoder' +str(i)))(enc_concat_attention)

        decoder_input = RepeatVector(y_train.shape[1],name='dec_repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('dec_alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('dec_attention_dot'+str(i)))
        attention = Activation('softmax', name='dec_attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='dec_Context'+str(i))
        dec_head_list.append(context)
    dec_concat_attention = Concatenate(axis=2)(dec_head_list)





    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)




    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)

    model = Model(inputs=input_train, outputs=out)

    model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['mae'])
    return model, model_checkpoint_callback, checkpoint_filepath

In [ ]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'db4', 2, 3)
    X_val= denoise_array(X_val, 0, 'db4', 2, 3)
    X_train = add_mp_reversed(X_train, mp_window)
    X_val = add_mp_reversed(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_wavelet1/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_wavelet1/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_wavelet1/val_loss.csv')
    np.save("2_Att_layer_MP_wavelet1/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_wavelet1/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 240s 2s/step - loss: 0.2764 - mae: 0.2668 - val_loss: 0.0539 - val_mae: 0.1521
Epoch 2/15
101/101 [==============================] - 154s 2s/step - loss: 0.2268 - mae: 0.2150 - val_loss: 0.0505 - val_mae: 0.1393
Epoch 3/15
101/101 [==============================] - 153s 2s/step - loss: 0.2236 - mae: 0.2178 - val_loss: 0.0480 - val_mae: 0.1379
Epoch 4/15
101/101 [==============================] - 153s 2s/step - loss: 0.2147 - mae: 0.2096 - val_loss: 0.0513 - val_mae: 0.1409
Epoch 5/15
101/101 [==============================] - 164s 2s/step - loss: 0.2085 - mae: 0.2049 - val_loss: 0.0533 - val_mae: 0.1456
Epoch 6/15
101/101 [==============================] - 170s 2s/step - loss: 0.2049 - mae: 0.2040 - val_loss: 0.0509 - val_mae: 0.1388
Epoch 7/15
101/101 [==============================] - 160s 2s/step - loss: 0.2028 - mae: 0.1992 - val_loss: 0.0522 - val_mae: 0.1456
Epoch 8/15
101/101 [==============================] - 154s 2s/step - 

In [20]:
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1


for month in [0]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train_D= denoise_array(X_train, 0, 'db4', 2, 3)
    X_val_D= denoise_array(X_val, 0, 'db4', 2, 3)
    
    
    

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


## MAE loss function


In [20]:
def LSTM_model(): 
    
    #set up callback for best val loss model
    checkpoint_filepath='./cnn/checkpoint'
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
    
    
    n_hidden = 30
    input_train = Input(shape=(n_steps_in, X_train.shape[2]),name='input')
    output_train = Input(shape=( y_train.shape[1], y_train.shape[2]))




    enc_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('encoder' +str(i)))(input_train)

        decoder_input = RepeatVector(y_train.shape[1], name='repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('attention_dot'+str(i)))
        attention = Activation('softmax', name='attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='Context'+str(i))
        enc_head_list.append(context)
    enc_concat_attention = Concatenate(axis=2)(enc_head_list)
    
    
    
    dec_head_list = []
    for i in range(0, X_train.shape[2]):
        encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
            n_hidden, activation='tanh', dropout=0.2, 
            return_state=True, return_sequences=True,name=('decoder' +str(i)))(enc_concat_attention)

        decoder_input = RepeatVector(y_train.shape[1],name='dec_repeat_vector'+str(i))(encoder_last_h)
        decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.2,
         return_state=False, return_sequences=True,name=('dec_alignment_model'+str(i)))(decoder_input, initial_state=[encoder_last_h, encoder_last_c])
        attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2], name=('dec_attention_dot'+str(i)))
        attention = Activation('softmax', name='dec_attention_activation'+str(i))(attention)
        context = dot([attention, encoder_stack_h], axes=[2,1],name='dec_Context'+str(i))
        dec_head_list.append(context)
    dec_concat_attention = Concatenate(axis=2)(dec_head_list)





    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)




    out = TimeDistributed(Dense(y_train.shape[2]))(dec_concat_attention)

    model = Model(inputs=input_train, outputs=out)

    model.compile(loss='MeanAbsoluteError', optimizer='Adam', metrics=['mse'])
    return model, model_checkpoint_callback, checkpoint_filepath

In [21]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'db4', 2, 3)
    X_val= denoise_array(X_val, 0, 'db4', 2, 3)
    X_train = add_mp_reversed(X_train, mp_window)
    X_val = add_mp_reversed(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_wavelet_mae/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_wavelet_mae/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_wavelet_mae/val_loss.csv')
    np.save("2_Att_layer_MP_wavelet_mae/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_wavelet_mae/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 231s 2s/step - loss: 0.2504 - mse: 0.2809 - val_loss: 0.1542 - val_mse: 0.0560
Epoch 2/15
101/101 [==============================] - 134s 1s/step - loss: 0.1991 - mse: 0.2358 - val_loss: 0.1544 - val_mse: 0.0519
Epoch 3/15
101/101 [==============================] - 134s 1s/step - loss: 0.1926 - mse: 0.2310 - val_loss: 0.1450 - val_mse: 0.0512
Epoch 4/15
101/101 [==============================] - 133s 1s/step - loss: 0.1879 - mse: 0.2279 - val_loss: 0.1699 - val_mse: 0.0588
Epoch 5/15
101/101 [==============================] - 133s 1s/step - loss: 0.1866 - mse: 0.2268 - val_loss: 0.1661 - val_mse: 0.0588
Epoch 6/15
101/101 [==============================] - 132s 1s/step - loss: 0.1833 - mse: 0.2234 - val_loss: 0.1681 - val_mse: 0.0622
Epoch 7/15
101/101 [==============================] - 133s 1s/step - loss: 0.1799 - mse: 0.2201 - val_loss: 0.1428 - val_mse: 0.0531
Epoch 8/15
101/101 [==============================] - 133s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 237s 1s/step - loss: 0.4292 - mse: 0.6828 - val_loss: 0.3672 - val_mse: 0.4377
Epoch 2/15
107/107 [==============================] - 140s 1s/step - loss: 0.3598 - mse: 0.5761 - val_loss: 0.3757 - val_mse: 0.4497
Epoch 3/15
107/107 [==============================] - 140s 1s/step - loss: 0.3468 - mse: 0.5559 - val_loss: 0.3799 - val_mse: 0.4558
Epoch 4/15
107/107 [==============================] - 140s 1s/step - loss: 0.3455 - mse: 0.5520 - val_loss: 0.3915 - val_mse: 0.4414
Epoch 5/15
107/107 [==============================] - 140s 1s/step - loss: 0.3408 - mse: 0.5436 - val_loss: 0.4110 - val_mse: 0.4890
Epoch 6/15
107/107 [==============================] - 140s 1s/step - loss: 0.3343 - mse: 0.5360 - val_loss: 0.5017 - val_mse: 0.5320
Epoch 7/15
107/107 [==============================] - 140s 1s/step - loss: 0.3346 - mse: 0.5336 - val_loss: 0.5334 - val_mse: 0.5788
Epoch 8/15
107/107 [==============================] - 140s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 226s 1s/step - loss: 0.2214 - mse: 0.1734 - val_loss: 0.2563 - val_mse: 0.3424
Epoch 2/15
101/101 [==============================] - 132s 1s/step - loss: 0.2016 - mse: 0.1573 - val_loss: 0.2337 - val_mse: 0.3474
Epoch 3/15
101/101 [==============================] - 132s 1s/step - loss: 0.1961 - mse: 0.1530 - val_loss: 0.2463 - val_mse: 0.3405
Epoch 4/15
101/101 [==============================] - 132s 1s/step - loss: 0.1931 - mse: 0.1503 - val_loss: 0.2654 - val_mse: 0.3343
Epoch 5/15
101/101 [==============================] - 131s 1s/step - loss: 0.1895 - mse: 0.1473 - val_loss: 0.2365 - val_mse: 0.3370
Epoch 6/15
101/101 [==============================] - 131s 1s/step - loss: 0.1856 - mse: 0.1442 - val_loss: 0.2627 - val_mse: 0.3336
Epoch 7/15
101/101 [==============================] - 131s 1s/step - loss: 0.1821 - mse: 0.1408 - val_loss: 0.2844 - val_mse: 0.3454
Epoch 8/15
101/101 [==============================] - 131s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
82/82 [==============================] - 205s 2s/step - loss: 0.2235 - mse: 0.2358 - val_loss: 0.1396 - val_mse: 0.0521
Epoch 2/15
82/82 [==============================] - 107s 1s/step - loss: 0.1394 - mse: 0.1632 - val_loss: 0.1453 - val_mse: 0.0442
Epoch 3/15
82/82 [==============================] - 107s 1s/step - loss: 0.1287 - mse: 0.1586 - val_loss: 0.1352 - val_mse: 0.0411
Epoch 4/15
82/82 [==============================] - 108s 1s/step - loss: 0.1259 - mse: 0.1556 - val_loss: 0.1313 - val_mse: 0.0455
Epoch 5/15
82/82 [==============================] - 107s 1s/step - loss: 0.1242 - mse: 0.1544 - val_loss: 0.1409 - val_mse: 0.0431
Epoch 6/15
82/82 [==============================] - 107s 1s/step - loss: 0.1223 - mse: 0.1537 - val_loss: 0.1349 - val_mse: 0.0443
Epoch 7/15
82/82 [==============================] - 106s 1s/step - loss: 0.1216 - mse: 0.1525 - val_loss: 0.1337 - val_mse: 0.0432
Epoch 8/15
82/82 [==============================] - 107s 1s/step - loss: 0.1178 - m

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 234s 2s/step - loss: 0.1839 - mse: 0.1262 - val_loss: 0.1420 - val_mse: 0.0435
Epoch 2/15
101/101 [==============================] - 132s 1s/step - loss: 0.1381 - mse: 0.0915 - val_loss: 0.1558 - val_mse: 0.0450
Epoch 3/15
101/101 [==============================] - 132s 1s/step - loss: 0.1355 - mse: 0.0869 - val_loss: 0.1336 - val_mse: 0.0391
Epoch 4/15
101/101 [==============================] - 132s 1s/step - loss: 0.1313 - mse: 0.0808 - val_loss: 0.1660 - val_mse: 0.0470
Epoch 5/15
101/101 [==============================] - 132s 1s/step - loss: 0.1287 - mse: 0.0763 - val_loss: 0.1900 - val_mse: 0.0544
Epoch 6/15
101/101 [==============================] - 132s 1s/step - loss: 0.1300 - mse: 0.0701 - val_loss: 0.1381 - val_mse: 0.0402
Epoch 7/15
101/101 [==============================] - 132s 1s/step - loss: 0.1218 - mse: 0.0619 - val_loss: 0.1479 - val_mse: 0.0424
Epoch 8/15
101/101 [==============================] - 131s 1s/step - 

## Bior wavelet, less aggresive denoising

In [22]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'bior3.3', 2, 10)
    X_val= denoise_array(X_val, 0, 'bior3.3', 2, 10)
    X_train = add_mp_reversed(X_train, mp_window)
    X_val = add_mp_reversed(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_wavelet2/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_wavelet2/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_wavelet2/val_loss.csv')
    np.save("2_Att_layer_MP_wavelet2/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_wavelet2/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 229s 1s/step - loss: 0.2707 - mae: 0.2650 - val_loss: 0.0475 - val_mae: 0.1416
Epoch 2/15
101/101 [==============================] - 129s 1s/step - loss: 0.2260 - mae: 0.2136 - val_loss: 0.0512 - val_mae: 0.1589
Epoch 3/15
101/101 [==============================] - 129s 1s/step - loss: 0.2224 - mae: 0.2128 - val_loss: 0.0729 - val_mae: 0.2013
Epoch 4/15
101/101 [==============================] - 131s 1s/step - loss: 0.2177 - mae: 0.2095 - val_loss: 0.0583 - val_mae: 0.1692
Epoch 5/15
101/101 [==============================] - 126s 1s/step - loss: 0.2132 - mae: 0.2083 - val_loss: 0.0515 - val_mae: 0.1499
Epoch 6/15
101/101 [==============================] - 125s 1s/step - loss: 0.2080 - mae: 0.2031 - val_loss: 0.0513 - val_mae: 0.1488
Epoch 7/15
101/101 [==============================] - 121s 1s/step - loss: 0.2021 - mae: 0.1989 - val_loss: 0.0543 - val_mae: 0.1518
Epoch 8/15
101/101 [==============================] - 122s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
107/107 [==============================] - 229s 1s/step - loss: 0.7073 - mae: 0.4874 - val_loss: 0.4192 - val_mae: 0.3936
Epoch 2/15
107/107 [==============================] - 130s 1s/step - loss: 0.5465 - mae: 0.3811 - val_loss: 0.4295 - val_mae: 0.4130
Epoch 3/15
107/107 [==============================] - 129s 1s/step - loss: 0.5307 - mae: 0.3755 - val_loss: 0.4166 - val_mae: 0.3611
Epoch 4/15
107/107 [==============================] - 129s 1s/step - loss: 0.5215 - mae: 0.3713 - val_loss: 0.4223 - val_mae: 0.3957
Epoch 5/15
107/107 [==============================] - 128s 1s/step - loss: 0.5150 - mae: 0.3719 - val_loss: 0.4582 - val_mae: 0.4045
Epoch 6/15
107/107 [==============================] - 129s 1s/step - loss: 0.4899 - mae: 0.3603 - val_loss: 0.4756 - val_mae: 0.4675
Epoch 7/15
107/107 [==============================] - 129s 1s/step - loss: 0.4808 - mae: 0.3621 - val_loss: 0.4808 - val_mae: 0.4604
Epoch 8/15
107/107 [==============================] - 129s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 217s 1s/step - loss: 0.1675 - mae: 0.2316 - val_loss: 0.3544 - val_mae: 0.3100
Epoch 2/15
101/101 [==============================] - 124s 1s/step - loss: 0.1493 - mae: 0.2083 - val_loss: 0.3471 - val_mae: 0.2360
Epoch 3/15
101/101 [==============================] - 123s 1s/step - loss: 0.1451 - mae: 0.2065 - val_loss: 0.3400 - val_mae: 0.2782
Epoch 4/15
101/101 [==============================] - 122s 1s/step - loss: 0.1382 - mae: 0.2011 - val_loss: 0.3463 - val_mae: 0.2590
Epoch 5/15
101/101 [==============================] - 122s 1s/step - loss: 0.1354 - mae: 0.1995 - val_loss: 0.3517 - val_mae: 0.2799
Epoch 6/15
101/101 [==============================] - 122s 1s/step - loss: 0.1294 - mae: 0.1951 - val_loss: 0.3533 - val_mae: 0.2612
Epoch 7/15
101/101 [==============================] - 122s 1s/step - loss: 0.1282 - mae: 0.1936 - val_loss: 0.3790 - val_mae: 0.3008
Epoch 8/15
101/101 [==============================] - 122s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
82/82 [==============================] - 197s 1s/step - loss: 0.2235 - mae: 0.2246 - val_loss: 0.0915 - val_mae: 0.2210
Epoch 2/15
82/82 [==============================] - 100s 1s/step - loss: 0.1611 - mae: 0.1512 - val_loss: 0.0476 - val_mae: 0.1347
Epoch 3/15
82/82 [==============================] - 99s 1s/step - loss: 0.1548 - mae: 0.1431 - val_loss: 0.0524 - val_mae: 0.1412
Epoch 4/15
82/82 [==============================] - 99s 1s/step - loss: 0.1545 - mae: 0.1465 - val_loss: 0.0392 - val_mae: 0.1246
Epoch 5/15
82/82 [==============================] - 98s 1s/step - loss: 0.1505 - mae: 0.1427 - val_loss: 0.0464 - val_mae: 0.1531
Epoch 6/15
82/82 [==============================] - 98s 1s/step - loss: 0.1447 - mae: 0.1350 - val_loss: 0.0416 - val_mae: 0.1241
Epoch 7/15
82/82 [==============================] - 99s 1s/step - loss: 0.1450 - mae: 0.1389 - val_loss: 0.0436 - val_mae: 0.1374
Epoch 8/15
82/82 [==============================] - 97s 1s/step - loss: 0.1428 - mae: 0.

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
101/101 [==============================] - 226s 1s/step - loss: 0.1188 - mae: 0.1891 - val_loss: 0.0781 - val_mae: 0.2120
Epoch 2/15
101/101 [==============================] - 122s 1s/step - loss: 0.0768 - mae: 0.1520 - val_loss: 0.0706 - val_mae: 0.2241
Epoch 3/15
101/101 [==============================] - 122s 1s/step - loss: 0.0645 - mae: 0.1410 - val_loss: 0.0783 - val_mae: 0.2413
Epoch 4/15
101/101 [==============================] - 122s 1s/step - loss: 0.0645 - mae: 0.1377 - val_loss: 0.0840 - val_mae: 0.2506
Epoch 5/15
101/101 [==============================] - 122s 1s/step - loss: 0.0575 - mae: 0.1340 - val_loss: 0.0601 - val_mae: 0.1976
Epoch 6/15
101/101 [==============================] - 122s 1s/step - loss: 0.0535 - mae: 0.1326 - val_loss: 0.0537 - val_mae: 0.1845
Epoch 7/15
101/101 [==============================] - 121s 1s/step - loss: 0.0552 - mae: 0.1354 - val_loss: 0.0696 - val_mae: 0.2153
Epoch 8/15
101/101 [==============================] - 121s 1s/step - 

## 12 hour MP window

In [23]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+144
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=144



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+144
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'bior3.3', 2, 10)
    X_val= denoise_array(X_val, 0, 'bior3.3', 2, 10)
    X_train = add_mp_reversed(X_train, mp_window)
    X_val = add_mp_reversed(X_val, mp_window)
    

    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=15, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_wavelet3/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_wavelet3/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_wavelet3/val_loss.csv')
    np.save("2_Att_layer_MP_wavelet3/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_wavelet3/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
104/104 [==============================] - 232s 1s/step - loss: 0.4024 - mae: 0.2867 - val_loss: 0.0521 - val_mae: 0.1549
Epoch 2/15
104/104 [==============================] - 125s 1s/step - loss: 0.3476 - mae: 0.2301 - val_loss: 0.0535 - val_mae: 0.1540
Epoch 3/15
104/104 [==============================] - 125s 1s/step - loss: 0.3448 - mae: 0.2294 - val_loss: 0.0549 - val_mae: 0.1554
Epoch 4/15
104/104 [==============================] - 125s 1s/step - loss: 0.3381 - mae: 0.2280 - val_loss: 0.0610 - val_mae: 0.1596
Epoch 5/15
104/104 [==============================] - 124s 1s/step - loss: 0.3265 - mae: 0.2269 - val_loss: 0.0567 - val_mae: 0.1486
Epoch 6/15
104/104 [==============================] - 125s 1s/step - loss: 0.3282 - mae: 0.2259 - val_loss: 0.0541 - val_mae: 0.1498
Epoch 7/15
104/104 [==============================] - 125s 1s/step - loss: 0.3148 - mae: 0.2218 - val_loss: 0.0542 - val_mae: 0.1543
Epoch 8/15
104/104 [==============================] - 124s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
111/111 [==============================] - 238s 1s/step - loss: 0.7107 - mae: 0.4845 - val_loss: 0.4200 - val_mae: 0.4063
Epoch 2/15
111/111 [==============================] - 134s 1s/step - loss: 0.5750 - mae: 0.3986 - val_loss: 0.4291 - val_mae: 0.3616
Epoch 3/15
111/111 [==============================] - 133s 1s/step - loss: 0.5570 - mae: 0.3898 - val_loss: 0.3941 - val_mae: 0.3603
Epoch 4/15
111/111 [==============================] - 133s 1s/step - loss: 0.5428 - mae: 0.3818 - val_loss: 0.4265 - val_mae: 0.3626
Epoch 5/15
111/111 [==============================] - 133s 1s/step - loss: 0.5260 - mae: 0.3784 - val_loss: 0.5372 - val_mae: 0.4716
Epoch 6/15
111/111 [==============================] - 133s 1s/step - loss: 0.5133 - mae: 0.3716 - val_loss: 0.3955 - val_mae: 0.3838
Epoch 7/15
111/111 [==============================] - 133s 1s/step - loss: 0.5079 - mae: 0.3745 - val_loss: 0.4006 - val_mae: 0.3673
Epoch 8/15
111/111 [==============================] - 133s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
104/104 [==============================] - 232s 1s/step - loss: 0.2677 - mae: 0.2510 - val_loss: 0.3427 - val_mae: 0.2421
Epoch 2/15
104/104 [==============================] - 126s 1s/step - loss: 0.2444 - mae: 0.2254 - val_loss: 0.3308 - val_mae: 0.2372
Epoch 3/15
104/104 [==============================] - 125s 1s/step - loss: 0.2388 - mae: 0.2262 - val_loss: 0.3346 - val_mae: 0.3004
Epoch 4/15
104/104 [==============================] - 125s 1s/step - loss: 0.2314 - mae: 0.2217 - val_loss: 0.3249 - val_mae: 0.2521
Epoch 5/15
104/104 [==============================] - 125s 1s/step - loss: 0.2261 - mae: 0.2216 - val_loss: 0.3277 - val_mae: 0.2477
Epoch 6/15
104/104 [==============================] - 125s 1s/step - loss: 0.2159 - mae: 0.2138 - val_loss: 0.3326 - val_mae: 0.2976
Epoch 7/15
104/104 [==============================] - 125s 1s/step - loss: 0.2116 - mae: 0.2094 - val_loss: 0.3573 - val_mae: 0.3382
Epoch 8/15
104/104 [==============================] - 125s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
85/85 [==============================] - 198s 1s/step - loss: 0.2409 - mae: 0.2114 - val_loss: 0.0470 - val_mae: 0.1315
Epoch 2/15
85/85 [==============================] - 103s 1s/step - loss: 0.1802 - mae: 0.1504 - val_loss: 0.0387 - val_mae: 0.1213
Epoch 3/15
85/85 [==============================] - 102s 1s/step - loss: 0.1765 - mae: 0.1513 - val_loss: 0.0408 - val_mae: 0.1416
Epoch 4/15
85/85 [==============================] - 102s 1s/step - loss: 0.1773 - mae: 0.1541 - val_loss: 0.0408 - val_mae: 0.1268
Epoch 5/15
85/85 [==============================] - 102s 1s/step - loss: 0.1691 - mae: 0.1419 - val_loss: 0.0516 - val_mae: 0.1855
Epoch 6/15
85/85 [==============================] - 103s 1s/step - loss: 0.1702 - mae: 0.1531 - val_loss: 0.0376 - val_mae: 0.1265
Epoch 7/15
85/85 [==============================] - 103s 1s/step - loss: 0.1625 - mae: 0.1415 - val_loss: 0.0426 - val_mae: 0.1328
Epoch 8/15
85/85 [==============================] - 102s 1s/step - loss: 0.1552 - m

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/15
104/104 [==============================] - 223s 1s/step - loss: 0.1248 - mae: 0.2048 - val_loss: 0.0652 - val_mae: 0.2089
Epoch 2/15
104/104 [==============================] - 125s 1s/step - loss: 0.0759 - mae: 0.1468 - val_loss: 0.0637 - val_mae: 0.1911
Epoch 3/15
104/104 [==============================] - 126s 1s/step - loss: 0.0588 - mae: 0.1350 - val_loss: 0.0644 - val_mae: 0.2013
Epoch 4/15
104/104 [==============================] - 123s 1s/step - loss: 0.0608 - mae: 0.1388 - val_loss: 0.0671 - val_mae: 0.2132
Epoch 5/15
104/104 [==============================] - 126s 1s/step - loss: 0.0588 - mae: 0.1344 - val_loss: 0.0486 - val_mae: 0.1678
Epoch 6/15
104/104 [==============================] - 125s 1s/step - loss: 0.0554 - mae: 0.1335 - val_loss: 0.0648 - val_mae: 0.2040
Epoch 7/15
104/104 [==============================] - 125s 1s/step - loss: 0.0521 - mae: 0.1309 - val_loss: 0.0628 - val_mae: 0.2015
Epoch 8/15
104/104 [==============================] - 125s 1s/step - 

In [24]:
keras.backend.clear_session()
#Create Training Examples for all lookaheads
resample_rate = '5T'
end_dates = ['2021-12-26 23:55:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00', '2022-04-26 23:55:00'  ]
Start_dates = ['2021-11-26 00:00:00', '2021-12-26 00:00:00', '2022-01-26 23:55:00', '2022-02-26 23:55:00', '2022-03-26 23:55:00']

#end_dates = ['2022-01-26 23:55:00', '2022-03-26 23:55:00']
#Start_dates = ['2021-11-26 00:00:00', '2022-01-26 00:00:00']
inputs = ['min_gas_price', 'block_gas_5th_percentile', 'block_gas_95th_percentile', 'gas_used', 'base_fee_per_gas', 'transaction_count', 'size', 'Open', 'contracts']
#No of timesteps behind to forecast on, no of timesteps to forecast ahead
n_steps_in = 4032+288
n_steps_out = 10
#How many timesteps between start of training examples
step_interval = 1
n_step_lookahead = 1
mp_window=288



y_hat_list=[]
train_loss_list=[]
val_loss_list=[]
training_metrics_dicts=[]
valdiation_metrics_dicts=[]
for month in [0, 1, 2, 3, 4]:
    n_steps_in = 4032+288
    n_step_lookahead = 1
    start_date=Start_dates[month]
    end_date=end_dates[month]
    X_train, y_train, X_val, y_val, scaler = generate_training_val_examples_univariate_output(data)
    X_train= denoise_array(X_train, 0, 'bior3.3', 2, 10)
    X_val= denoise_array(X_val, 0, 'bior3.3', 2, 10)
    X_train = add_mp_reversed(X_train, mp_window)
    X_val = add_mp_reversed(X_val, mp_window)
    
   
    
    n_steps_in = 4032
    
    
    model, model_checkpoint_callback, checkpoint_filepath = LSTM_model()
    train_history = model.fit(X_train, y_train,validation_data=(X_val, y_val), epochs=35, verbose=1, callbacks=[model_checkpoint_callback])
    model.load_weights(checkpoint_filepath)
    train_loss_list.append(train_history.history['loss'])
    val_loss_list.append(train_history.history['val_loss'])
    model.save_weights(('2_Att_layer_MP_wavelet4/Month' +str(month)))
    
    yhat_train=model.predict(X_train, verbose=1)
    yhat_val = model.predict(X_val, verbose=1)  
    training_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_train, y_train))
    valdiation_metrics_dicts.append(descale_y_retrun_metrics_univariate_y(yhat_val, y_val))
    pd.DataFrame(train_loss_list).to_csv('2_Att_layer_MP_wavelet4/train_loss.csv')
    pd.DataFrame(val_loss_list).to_csv('2_Att_layer_MP_wavelet2/val_loss.csv')
    np.save("2_Att_layer_MP_wavelet4/training_metrics.npy", training_metrics_dicts)
    np.save("2_Att_layer_MP_wavelet4/val_metrics.npy", valdiation_metrics_dicts)

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/35
101/101 [==============================] - 223s 1s/step - loss: 0.2786 - mae: 0.2731 - val_loss: 0.0565 - val_mae: 0.1690
Epoch 2/35
101/101 [==============================] - 122s 1s/step - loss: 0.2299 - mae: 0.2209 - val_loss: 0.0561 - val_mae: 0.1596
Epoch 3/35
101/101 [==============================] - 122s 1s/step - loss: 0.2192 - mae: 0.2077 - val_loss: 0.0551 - val_mae: 0.1530
Epoch 4/35
101/101 [==============================] - 122s 1s/step - loss: 0.2133 - mae: 0.2100 - val_loss: 0.0708 - val_mae: 0.1875
Epoch 5/35
101/101 [==============================] - 122s 1s/step - loss: 0.2063 - mae: 0.2049 - val_loss: 0.0516 - val_mae: 0.1422
Epoch 6/35
101/101 [==============================] - 122s 1s/step - loss: 0.2037 - mae: 0.1999 - val_loss: 0.0541 - val_mae: 0.1508
Epoch 7/35
101/101 [==============================] - 122s 1s/step - loss: 0.2048 - mae: 0.2009 - val_loss: 0.0567 - val_mae: 0.1577
Epoch 8/35
101/101 [==============================] - 122s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/35
107/107 [==============================] - 224s 1s/step - loss: 0.7468 - mae: 0.5055 - val_loss: 0.4229 - val_mae: 0.3766
Epoch 2/35
107/107 [==============================] - 128s 1s/step - loss: 0.5521 - mae: 0.3851 - val_loss: 0.3801 - val_mae: 0.3618
Epoch 3/35
107/107 [==============================] - 127s 1s/step - loss: 0.5350 - mae: 0.3797 - val_loss: 0.4156 - val_mae: 0.3734
Epoch 4/35
107/107 [==============================] - 128s 1s/step - loss: 0.5240 - mae: 0.3751 - val_loss: 0.4208 - val_mae: 0.3718
Epoch 5/35
107/107 [==============================] - 128s 1s/step - loss: 0.5229 - mae: 0.3744 - val_loss: 0.4100 - val_mae: 0.4135
Epoch 6/35
107/107 [==============================] - 128s 1s/step - loss: 0.5025 - mae: 0.3696 - val_loss: 0.4372 - val_mae: 0.4398
Epoch 7/35
107/107 [==============================] - 128s 1s/step - loss: 0.4851 - mae: 0.3627 - val_loss: 0.4265 - val_mae: 0.4030
Epoch 8/35
107/107 [==============================] - 128s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/35
101/101 [==============================] - 222s 1s/step - loss: 0.1634 - mae: 0.2254 - val_loss: 0.3507 - val_mae: 0.2367
Epoch 2/35
101/101 [==============================] - 121s 1s/step - loss: 0.1542 - mae: 0.2149 - val_loss: 0.3381 - val_mae: 0.2697
Epoch 3/35
101/101 [==============================] - 122s 1s/step - loss: 0.1480 - mae: 0.2088 - val_loss: 0.3463 - val_mae: 0.2329
Epoch 4/35
101/101 [==============================] - 121s 1s/step - loss: 0.1426 - mae: 0.2033 - val_loss: 0.3514 - val_mae: 0.2478
Epoch 5/35
101/101 [==============================] - 121s 1s/step - loss: 0.1379 - mae: 0.2025 - val_loss: 0.3498 - val_mae: 0.2723
Epoch 6/35
101/101 [==============================] - 122s 1s/step - loss: 0.1297 - mae: 0.1949 - val_loss: 0.3542 - val_mae: 0.2725
Epoch 7/35
101/101 [==============================] - 121s 1s/step - loss: 0.1262 - mae: 0.1907 - val_loss: 0.3545 - val_mae: 0.2563
Epoch 8/35
101/101 [==============================] - 121s 1s/step - 

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/35
82/82 [==============================] - 201s 1s/step - loss: 0.2033 - mae: 0.2059 - val_loss: 0.0994 - val_mae: 0.2467
Epoch 2/35
82/82 [==============================] - 103s 1s/step - loss: 0.1600 - mae: 0.1551 - val_loss: 0.0682 - val_mae: 0.1807
Epoch 3/35
82/82 [==============================] - 103s 1s/step - loss: 0.1576 - mae: 0.1517 - val_loss: 0.0435 - val_mae: 0.1305
Epoch 4/35
82/82 [==============================] - 102s 1s/step - loss: 0.1549 - mae: 0.1472 - val_loss: 0.0405 - val_mae: 0.1345
Epoch 5/35
82/82 [==============================] - 101s 1s/step - loss: 0.1500 - mae: 0.1424 - val_loss: 0.0416 - val_mae: 0.1248
Epoch 6/35
82/82 [==============================] - 101s 1s/step - loss: 0.1484 - mae: 0.1382 - val_loss: 0.0397 - val_mae: 0.1252
Epoch 7/35
82/82 [==============================] - 102s 1s/step - loss: 0.1465 - mae: 0.1394 - val_loss: 0.0425 - val_mae: 0.1424
Epoch 8/35
82/82 [==============================] - 101s 1s/step - loss: 0.1420 - m

D:\Anaconda\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


Epoch 1/35
101/101 [==============================] - 230s 1s/step - loss: 0.1110 - mae: 0.1857 - val_loss: 0.0683 - val_mae: 0.1976
Epoch 2/35
101/101 [==============================] - 125s 1s/step - loss: 0.0719 - mae: 0.1470 - val_loss: 0.0521 - val_mae: 0.1638
Epoch 3/35
101/101 [==============================] - 125s 1s/step - loss: 0.0693 - mae: 0.1462 - val_loss: 0.0651 - val_mae: 0.1870
Epoch 4/35
101/101 [==============================] - 125s 1s/step - loss: 0.0599 - mae: 0.1347 - val_loss: 0.0629 - val_mae: 0.2015
Epoch 5/35
101/101 [==============================] - 126s 1s/step - loss: 0.0572 - mae: 0.1342 - val_loss: 0.0705 - val_mae: 0.2179
Epoch 6/35
101/101 [==============================] - 125s 1s/step - loss: 0.0583 - mae: 0.1365 - val_loss: 0.0820 - val_mae: 0.2380
Epoch 7/35
101/101 [==============================] - 126s 1s/step - loss: 0.0553 - mae: 0.1311 - val_loss: 0.0440 - val_mae: 0.1532
Epoch 8/35
101/101 [==============================] - 127s 1s/step - 